In [21]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim

In [22]:
# For reproducibility 재현성(똑같은 결과를 재현)
torch.manual_seed(1)


In [23]:
x_data = [[1,2],[2,3],[3,1],[4,3],[5,3],[6,2]]
y_data = [[0],[0],[0],[1],[1],[1]]

In [24]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [25]:
print(x_train)
print(y_train)

tensor([[1., 2.],
        [2., 3.],
        [3., 1.],
        [4., 3.],
        [5., 3.],
        [6., 2.]])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


In [26]:
print(x_train.shape)
print(y_train.shape)

torch.Size([6, 2])
torch.Size([6, 1])


In [27]:
# pytorch has a torch.exp() function that resembles the exponential funtion
print('e^1 equals:', torch.exp(torch.FloatTensor([1])))

e^1 equals: tensor([2.7183])


In [28]:
W = torch.zeros((2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [29]:
hypothesis = 1 / (1+torch.exp(-x_train.matmul(W)+b))

In [30]:
print(hypothesis)
print(hypothesis.shape)
print(y_train)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


In [31]:
#For one element, the loss can be computed as follows:
-(y_train[0]*torch.log(hypothesis[0])+(1-y_train[0])*torch.log(1-hypothesis[0]))
-(y_train[1]*torch.log(hypothesis[1])+(1-y_train[1])*torch.log(1-hypothesis[1]))

tensor([0.6931], grad_fn=<NegBackward>)

In [32]:
losses = -(y_train*torch.log(hypothesis)+(1-y_train)*torch.log(1-hypothesis))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward>)


In [33]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


In [34]:
F.binary_cross_entropy(hypothesis, y_train) #  = losses

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward>)

In [78]:
# 모델 초기화
W = torch.zeros((2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W,b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    #cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W)+b)
    
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    #100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} cost: {:.6f}'.format(epoch,nb_epochs, cost.item()))
    
    
    
    
    
    

Epoch    0/1000 cost: 0.693147
Epoch  100/1000 cost: 0.134722
Epoch  200/1000 cost: 0.080643
Epoch  300/1000 cost: 0.057900
Epoch  400/1000 cost: 0.045300
Epoch  500/1000 cost: 0.037261
Epoch  600/1000 cost: 0.031673
Epoch  700/1000 cost: 0.027556
Epoch  800/1000 cost: 0.024394
Epoch  900/1000 cost: 0.021888
Epoch 1000/1000 cost: 0.019852


In [79]:
hypothesis = torch.sigmoid(x_train.matmul(W)+b)
print(hypothesis[:5])


tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01]], grad_fn=<SliceBackward>)


In [80]:
prediction = hypothesis >= torch.FloatTensor([0.5])
print(prediction)

tensor([[False],
        [False],
        [False],
        [ True],
        [ True],
        [ True]])


In [89]:
correct_prediction= prediction.float() == y_train.float()
print(correct_prediction[:5].float())


tensor([[0.],
        [0.],
        [1.],
        [1.],
        [0.]])


In [90]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2,1) # W
        self.sigmoid = nn.Sigmoid() # b 
    
    def forward(self,x):
        return self.sigmoid(self.linear(x))

In [91]:
model= BinaryClassifier()


In [98]:
# Optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

nb_epochs = 100 
for epoch in range(nb_epochs + 1):
    
    # H(X)계산
    hypothesis = model(x_train)
    
    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction= prediction.float() == y_train.float()
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} cost: {:.6f},Accuracy {:.6f}%'.format(epoch,nb_epochs, cost.item(), accuracy * 100))

Epoch    0/100 cost: 0.044383,Accuracy 100.000000%
Epoch   10/100 cost: 0.043462,Accuracy 100.000000%
Epoch   20/100 cost: 0.042579,Accuracy 100.000000%
Epoch   30/100 cost: 0.041732,Accuracy 100.000000%
Epoch   40/100 cost: 0.040918,Accuracy 100.000000%
Epoch   50/100 cost: 0.040136,Accuracy 100.000000%
Epoch   60/100 cost: 0.039384,Accuracy 100.000000%
Epoch   70/100 cost: 0.038660,Accuracy 100.000000%
Epoch   80/100 cost: 0.037963,Accuracy 100.000000%
Epoch   90/100 cost: 0.037291,Accuracy 100.000000%
Epoch  100/100 cost: 0.036642,Accuracy 100.000000%
